In [1]:
import requests
import json
from web3 import Web3
import asyncio
import websockets

import pandas as pd

In [2]:
### Uniswap v2 ###
# Addresses
UNISWAP_V2_FACTORY_ADDRESS = "0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f"
# Subgraphs
UNISWAP_V2_HTTP_URI = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2"
UNISWAP_V2_WSS_URI = "wss://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2"

### Uniswap v3 ###
# Addresses
UNISWAP_V3_FACTORY_ADDRESS = "0x1F98431c8aD98523631AE4a59f267346ea31F984"
UNISWAP_V3_MULTICALL2_ADDRESS = "0x5BA1e12693Dc8F9c48aAD8770482f4739bEeD696"
UNISWAP_V3_PROXYADMIN_ADDRESS = "0xB753548F6E010e7e680BA186F9Ca1BdAB2E90cf2"
UNISWAP_V3_TICKLENS_ADDRESS = "0xbfd8137f7d1516D3ea5cA83523914859ec47F573"
UNISWAP_V3_QUOTER_ADDRESS = "0xb27308f9F90D607463bb33eA1BeBb41C27CE5AB6"
UNISWAP_V3_SWAPROUTER_ADDRESS = "0xE592427A0AEce92De3Edee1F18E0157C05861564"
UNISWAP_V3_SWAPROUTER02_ADDRESS = "0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45"
UNISWAP_V3_NFTDESCRIPTIOR_ADDRESS = "0x42B24A95702b9986e82d421cC3568932790A48Ec"
UNISWAP_V3_NONFUNGIBLETOKENPOSITIONDESCRIPTOR_ADDRESS = "0x91ae842A5Ffd8d12023116943e72A606179294f3"
UNISWAP_V3_TRANSPARENTUPGRADABLEPROXY_ADDRESS = "0xb27308f9F90D607463bb33eA1BeBb41C27CE5AB6"
UNISWAP_V3_NONFUNGIBLEPOSITIONMANAGER_ADDRESS = "0xC36442b4a4522E871399CD717aBDD847Ab11FE88"
UNISWAP_V3_V3MIGRATOR_ADDRESS = "0xC36442b4a4522E871399CD717aBDD847Ab11FE88"
UNISWAP_V3_UNISWAPV3STAKER_ADDRESS = "0xe34139463bA50bD61336E0c446Bd8C0867c6fE65"
# Subgraphs
UNISWAP_V3_HTTP_URI = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"
UNISWAP_V3_WSS_URI = "wss://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"
# Additional info
UNISWAP_V3_FEE_TIERS = {
    "1%": 10000,
    "0.3%": 3000,
    "0.05%": 500,
    "0.01%": 100
}
# ABI
FACTORY_ABI_PATH = "../abi/uniswapv3/factory.json" 
POOL_ABI_PATH = "../abi/uniswapv3/pool.json"
with open(FACTORY_ABI_PATH, "r") as json_file:
    FACTORY_ABI = json.load(json_file)
with open(POOL_ABI_PATH, "r") as json_file:
    POOL_ABI = json.load(json_file)

In [3]:
### Query Templates ###
POOL_QUERY = """
{
  pools (first: <<first>> skip: <<skip>>) {
    id
    createdAtTimestamp
    createdAtBlockNumber
    token0 {
      id
      symbol
      name
    }
    token1 {
      id
      symbol
      name
    }
    feeTier
    liquidity
    sqrtPrice
    feeGrowthGlobal0X128
    feeGrowthGlobal1X128
    token0Price
    token1Price
    tick
    observationIndex
    volumeToken0
    volumeToken1
    volumeUSD
    untrackedVolumeUSD
    feesUSD
    txCount
    collectedFeesToken0
    collectedFeesToken1
    collectedFeesUSD
    totalValueLockedToken0
    totalValueLockedToken1
    totalValueLockedETH
    totalValueLockedUSD
    totalValueLockedUSDUntracked
    liquidityProviderCount
    poolHourData
    poolDayData
    mints
    burns
    swaps
    collects
    ticks
  }
}
"""

TOKEN_QUERY = """
{
  tokens (first: <<first>> skip: <<skip>>) {
    id
    symbol
    name
    decimals
    totalSupply
    volume
    volumeUSD
    untrackedVolumeUSD
    feesUSD
    txCount
    poolCount
    totalValueLocked
    totalValueLockedUSD
    totalValueLockedUSDUntracked
    derivedETH
    whitelistPools
    tokenDayData
  }
}
"""

In [4]:
def query_all(endpoint, query_string, num_results_per_query):
    assert num_results_per_query <= 1000, "Max allowed results per query for graphQL == 1000!"
    in_progress = True
    num_skip = 0
    output = {}
    while in_progress:
        complete_query_string = query_string.replace("<<first>>", str(num_results_per_query)).replace("<<skip>>", str(num_skip))
        query_json = {"query": complete_query_string}
        response = requests.post(endpoint, json=query_json)
        results = response.json()
        if "errors" in results:
            in_progress = False
        else:
            results_data = results["data"]
            for query_name in results_data:
                if query_name not in output:
                    output[query_name] = []
                for row in results_data[query_name]:
                    output[query_name].append(row)
            num_skip += num_results_per_query
    return output


def unpack_token_info_in_pool(token_dict):
    token_id = token_dict["id"]
    token_symbol = token_dict["symbol"]
    token_name = token_dict["name"]
    return token_id, token_symbol, token_name

In [5]:
tokens_details = query_all(UNISWAP_V3_HTTP_URI, TOKEN_QUERY, 1000)
pools_details = query_all(UNISWAP_V3_HTTP_URI, POOL_QUERY, 1000)

In [6]:
token_df = pd.DataFrame(tokens_details["tokens"])
pool_df = pd.DataFrame(pools_details["pools"])

token_df = token_df.infer_objects()
pool_df = pool_df.infer_objects()

token_convert_dict = {
    'decimals': float,
    'totalSupply': float,
    'volume': float,
    'volumeUSD': float,
    'feesUSD': float,
    'txCount': float,
    'poolCount': float,
    'totalValueLocked': float,
    'totalValueLockedUSD': float
}

token_df = token_df.astype(token_convert_dict)

pool_df["token0_id"] = pool_df.token0.map(lambda x : x["id"])
pool_df["token0_symbol"] = pool_df.token0.map(lambda x : x["symbol"])
pool_df["token0_name"] = pool_df.token0.map(lambda x : x["name"])
pool_df["token1_id"] = pool_df.token1.map(lambda x : x["id"])
pool_df["token1_symbol"] = pool_df.token1.map(lambda x : x["symbol"])
pool_df["token1_name"] = pool_df.token1.map(lambda x : x["name"])
pool_df = pool_df.drop(columns=["token0", "token1"])

In [7]:
token_df

,id,symbol,name,decimals,totalSupply,volume,volumeUSD,untrackedVolumeUSD,feesUSD,txCount,poolCount,totalValueLocked,totalValueLockedUSD,totalValueLockedUSDUntracked,derivedETH
0,0x00000000000045166c45af0fc6e4cf31d9e14b9a,BID,TopBidder,18.0,28240.0,8.347642e+03,8.114480e+03,4057.240035163701193523059301160956,81.144801,15.0,0.0,4.569066e+01,0.000000e+00,0,0
1,0x0000000000004946c0e9f43f4dee607b0ef1fa1c,CHI,Chi Gastoken by 1inch,0.0,28240.0,2.297910e+05,1.283028e+05,116375.2899007343362783226704308962,1278.926164,131.0,0.0,3.918400e+04,0.000000e+00,0,0
2,0x0000000000085d4780b73119b644ae5ecd22b376,TUSD,TrueUSD,18.0,18240.0,1.672619e+08,1.620988e+08,161780954.1113098592478457371590133,40422.011008,3872.0,0.0,1.183849e+07,1.183849e+07,0,0.0003482790234962949936430929505947498
3,0x0000000000095413afc295d19edeb1ad7b71c952,LON,Tokenlon,18.0,28240.0,1.627850e+03,7.395446e+03,3697.722887375783837488189918277345,22.186337,39.0,0.0,3.095965e+02,0.000000e+00,0,0
4,0x000000000ca5171087c18fb271ca844a2370fc0a,MERKLE,Merkle Network Token,18.0,18240.0,0.000000e+00,0.000000e+00,0,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4579,0xff89942e645271bf0b6756a6064232986a754b7d,NFTB,NFTB,18.0,18240.0,1.635783e+02,1.000804e+00,0.5004017677334099359738103484439915,0.010008,2.0,0.0,9.999984e+07,0.000000e+00,0,0
4580,0xff9255ae90134ec1e33588788ca7fecbee411340,Doge,Dogecoin,18.0,28240.0,0.000000e+00,0.000000e+00,0,0.000000,2.0,0.0,1.000000e-18,0.000000e+00,0,0
4581,0xffbf315f70e458e49229654dea4ce192d26f9b25,VOLT,VOLTAGE,18.0,28240.0,2.350362e+07,1.277027e+07,9686116.923076616063799167400368411,127702.719691,8377.0,0.0,1.422435e+06,0.000000e+00,0,0
4582,0xffe1ab085332720f63cf6784b94b43f45ee24c8f,DXB,Dubai Expo Ring,18.0,28240.0,5.693406e+05,1.305905e+02,65.29527099378633927474110776752684,1.305905,4.0,0.0,1.194283e+07,0.000000e+00,0,0


In [8]:
pool_df

,id,createdAtTimestamp,createdAtBlockNumber,feeTier,liquidity,sqrtPrice,feeGrowthGlobal0X128,feeGrowthGlobal1X128,token0Price,token1Price,...,totalValueLockedETH,totalValueLockedUSD,totalValueLockedUSDUntracked,liquidityProviderCount,token0_id,token0_symbol,token0_name,token1_id,token1_symbol,token1_name
0,0x0001fcbba8eb491c3ccfeddc5a5caba1a98c4c28,1626494775,12842087,10000,303015134493562686441,792216481398733702759960397,11229880233198962195483552111498235,0,10001.64466212999659279505112078187,0.00009998355608316876218708990103803189,...,0.01365551549733099,25.51011790401229376312534072638212,0,0,0xbef81556ef066ec840a540595c8d12f516b6378f,BCZ,BinanceCZ,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether
1,0x0002e63328169d7feea121f1e32e4f620abf0352,1647291448,14387035,3000,3138730438279534226,4796767726308887992116367,97940430690667720081687185988369629146540,261813467188232677565899279926960,0.2728109445182117690597702179726835,3.665542090937792248144112431103038,...,513.8995607463123014536952883885031,1472843.982736720381981149044642668,0,0,0x0d438f3b5175bebc262bf23753c1e53d03432bde,wNXM,Wrapped NXM,0x903bef1736cddf2a537176cf3c64579c3867a881,ICHI,ichi.farm
2,0x000ea4a83acefdd62b1b43e9ccc281f442651520,1620289511,12379648,3000,0,1525445232124877880812669990,679190941978595037524533653564822571915,248972682810178986777692163811709741,2697.527791994638524424494441909943,0.0003707098043503633165020695797911745,...,4.470719996186258815939922530903531,12361.79532514259019018172745788657,0,0,0x4fabb145d64652a948d72533023f6e7a623c7c53,BUSD,Binance USD,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether
3,0x000f0c0b0b791e855dcc5ad6501c7529dea882e0,1623133218,12592237,3000,0,0,0,0,0,0,...,0.002199999999992589,5.50453119311343319477446237369603,0,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether,0xd1063ee5ec2891991a29fefb52bcc448cd386844,DOGGE,BanDogge Mastiff
4,0x0025ade782cc2b2415d1e841a8d52ff5dce33dfe,1622550974,12548595,3000,292918389273214376584,513343315021712300679836410313039,4486230789878839738362799491084544,20577687480327188608432617665550293986944,0.00000002382008713586316139465919193438618,41981374.55569653180400908165970594,...,1.265986061592116398,3443.677841278021411735024117953202,0,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether,0xea5b19c073f8074f9a8dcd693d01b6acb1c26ae4,SIM,Simba Inu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0xeb100c7a89e4a002f4f0e19bc9770f04572da192,1648506168,14477342,3000,131693912511603987568616,2459907515849059981327986292801,56715202239905632047826864849218594,46270111242149289150571661584658068772,0.001037341155989312303696070245159081,964.0030131131738992145288176258701,...,204.2097964069181563442493017379721,602608.0954426999404649756801852728,0,0,0x6810e776880c02933d47db1b9fc05908e5386b96,GNO,Gnosis Token,0xdef1ca1fb7fbcdc777520aa7f396b4e015f497ab,COW,CoW Protocol Token
5996,0xeb121eda08b6fa7e8e1d242bcc970fd6227734b8,1623849437,12645707,3000,4169271386955331330952,557996579335175327961110573,490837843974201178012929062711585098,0,20160.25839907636490344876812000409,0.00004960253882687409630317764597942884,...,0.1555702873848723494646413161493254,288.1472985595347460134575441046479,0,0,0x3a810ff7211b40c4fa76205a14efe161615d0385,AIN,AI Network,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether
5997,0xeb22cab10d66f9c66568e348d2f51c94e5a20d8e,1622783440,12566001,500,0,71968380940647629610420477038,1638983682377504093695660235067907,1339608835076873097102021137245941,1.211924882893192429189189582391747,0.8251336482280399886479587324000135,...,0,0,0,0,0x0a5e677a6a24b2f1a2bf4f3bffc443231d2fdec8,USX,dForce USD,0xb986f3a2d91d3704dc974a24fb735dcc5e3c1e70,EUX,dForce EUR
5998,0xeb23039d6542a413839f5a9e1e8df16563e1d0fb,1638440875,13726836,10000,0,34103113950248496219019760632929157,14070040070316906632834051448272,2962541072051448147261906728632215752432595,5.397232106083903856237740881477156,

In [9]:
pool_df.loc[pool_df.token0_symbol == "USDC"].nlargest(20, "liquidity")

TypeError: Column 'liquidity' has dtype object, cannot use method 'nlargest' with this dtype

In [10]:
a = Web3.toChecksumAddress("0xbef81556ef066ec840a540595c8d12f516b6378f")
b = Web3.toChecksumAddress("0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2")
factory_contract.functions.getPool(a, b, UNISWAP_V3_FEE_TIERS["0.01%"]).call()

NameError: name 'factory_contract' is not defined

In [11]:
def handle_event(event):
    print(Web3.toJSON(event))
    # and whatever

# asynchronous defined function to loop
# this loop sets up an event filter and is looking for new entires for the "PairCreated" event
# this loop runs on a poll interval
async def log_loop(event_filter, poll_interval):
    while True:
        for PairCreated in event_filter.get_new_entries():
            handle_event(PairCreated)
        await asyncio.sleep(poll_interval)

# when main is called
# create a filter for the latest block and look for the "PairCreated" event for the uniswap factory contract
# run an async loop
# try to run the log_loop function above every 2 seconds
def main():
    event_filter = contract.events.PairCreated.createFilter(fromBlock='latest')
    #block_filter = web3.eth.filter('latest')
    # tx_filter = web3.eth.filter('pending')
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(
            asyncio.gather(
                log_loop(event_filter, 2)))
                # log_loop(block_filter, 2),
                # log_loop(tx_filter, 2)))
    finally:
        # close loop to free up system resources
        loop.close()

In [12]:
# this program queries for all of the uniswap pair addresses and their token supply
infura_https_url = "https://mainnet.infura.io/v3/4342e0f8305947849c8c30344c2ed900"
infura_wss_url = "wss://mainnet.infura.io/ws/v3/4342e0f8305947849c8c30344c2ed900"
# web3 = Web3(Web3.HTTPProvider(infura_https_url, request_kwargs={"timeout": 60}))
web3 = Web3(
    Web3.WebsocketProvider(infura_wss_url)
)

# Uniswap v3 factory
factory_contract = web3.eth.contract(
    address=UNISWAP_V3_FACTORY_ADDRESS,
    abi=FACTORY_ABI
)
pool_contract = web3.eth.contract(
    address=Web3.toChecksumAddress("0x0001fcbba8eb491c3ccfeddc5a5caba1a98c4c28"),
    abi=POOL_ABI
)

In [13]:
factory_contract.events.PoolCreated.createFilter(fromBlock='latest')

In [76]:
pool_contract.events.

[0, 0, 0, False]

In [13]:
event_filter = pool_contract.events.Swap.createFilter(fromBlock='latest')